**You might need to restart runtime after running the below code, so that the right version of tensorflow is ready for use. Please check the output of the code to see if a restart is needed.**

In [ ]:
!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 92.5MB 1.3MB/s 
     |████████████████████████████████| 3.2MB 47.1MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 368kB 51.3MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 345.2MB 36kB/s 


In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

True

In [ ]:
#pysaliency is a library to extract fixations data from a set of mat files
#And compute saliency metrics for models

!git clone https://github.com/matthias-k/pysaliency.git 
!pip install ./pysaliency

import pysaliency
import sys
sys.modules[__name__] = pysaliency

Cloning into 'pysaliency'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 1804 (delta 99), reused 80 (delta 39), pack-reused 1655
Receiving objects: 100% (1804/1804), 139.34 MiB | 35.65 MiB/s, done.
Resolving deltas: 100% (1247/1247), done.
Processing ./pysaliency
     |████████████████████████████████| 174kB 13.4MB/s 
  Created wheel for pysaliency: filename=pysaliency-0.2.20-cp36-cp36m-linux_x86_64.whl size=335763 sha256=c50f7de1c59804d9a6064ce20569e33b41dd37dad8e0a1ffb4dbc48d9746694e
  Stored in directory: /tmp/pip-ephem-wheel-cache-clhiz_4m/wheels/41/88/d3/f9b878cf41eb198d3cdd33d2408b743c55342b053fb1f0a4a1
Successfully built pysaliency


**Download MIT1003 dataset for validation and testing**

In [ ]:
                                                                                     
#MIT1003 dataset for fine tuning the readout network

import zipfile
from .datasets import FileStimuli, Stimuli, FixationTrains, Fixations, read_hdf5
import os 
import glob
import numpy as np
from natsort import natsorted
import shutil

def create_stimuli(stimuli_location, filenames, location=None, attributes=None):
    """
    Create a Stimuli class of stimuli.
    Parameters
    ----------
    @type  stimuli_location: string
    @param stimuli_location: the base path where the stimuli are located.
                             If `location` is provided, this directory will
                             be copied to `location` (see below).
    @type  filenames: list of strings
    @param filenames: lists the filenames of the stimuli to include in the dataset.
                      Filenames have to be relative to `stimuli_location`.
    @type  location: string or `None`
    @param location: If provided, the function will copy the filenames to
                     `location` and return a `FileStimuli`-object for the
                     copied files. Otherwise a `Stimuli`-object is returned.
    @returns: `Stimuli` or `FileStimuli` object depending on `location`.
    """
    if location is not None:
        shutil.copytree(stimuli_location,
                        location)
        filenames = [os.path.join(location, f) for f in filenames]

        return FileStimuli(filenames, attributes=attributes)

    else:
        filenames = [os.path.join(stimuli_location, f) for f in filenames]
        return create_memory_stimuli(filenames, attributes=attributes)

!wget http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLSTIMULI.zip 
!wget http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLFIXATIONMAPS.zip 

!mkdir "/content/mit"
!mkdir "/content/mit/saliencymap/"


def filter_files(filenames, ignores):
    """
    Filter a list of files, excluding all filenames which contain
    an element of `ignores` as part of their path
    """
    parts = map(full_split, filenames)
    inds = [i for i, ps in enumerate(parts)
            if not any([ignore in ps for ignore in ignores])]
    return [filenames[i] for i in inds]

def full_split(filename):
    """ split filename into all parts """
    path, basename = os.path.split(filename)
    components = [path, basename]
    while components[0] and components[0] != '/':
        first_part = components.pop(0)
        path, basename = os.path.split(first_part)
        components = [path, basename] + components

    if components[0] == '':
        components = components[1:]

    return components   

def which(program):
    """
    Check whether a program is present on the system.
    from https://stackoverflow.com/a/377028
    """
    def is_exe(fpath):
        return os.path.isfile(fpath) and os.access(fpath, os.X_OK)

    fpath, fname = os.path.split(program)
    if fpath:
        if is_exe(program):
            return program
    else:
        for path in os.environ["PATH"].split(os.pathsep):
            path = path.strip('"')
            exe_file = os.path.join(path, program)
            if is_exe(exe_file):
                return exe_file

    return None

# MIT Stimuli

def MIT_stimuli(zip_name):
    temp_dir = './'
    print('Creating stimuli')
    f = zipfile.ZipFile(os.path.join(temp_dir, zip_name))
    namelist = f.namelist()
    namelist = filter_files(namelist, ['.svn', '__MACOSX', '.DS_Store'])
    f.extractall(temp_dir, namelist)

    stimuli_src_location = os.path.join(temp_dir, 'ALLSTIMULI')
    stimuli_target_location = os.path.join('./mit/', 'stimuli')
    images = glob.glob(os.path.join(stimuli_src_location, '*.jpeg'))

    images = [os.path.split(img)[1] for img in images]

    mit_stimuli_filenames = natsorted(images)

    mit_stimuli = create_stimuli(stimuli_src_location, mit_stimuli_filenames, stimuli_target_location)

    return mit_stimuli , mit_stimuli_filenames

mit_stimuli , mit_stimuli_filenames = MIT_stimuli('ALLSTIMULI.zip')

os.remove("./ALLSTIMULI.zip")

with zipfile.ZipFile('./ALLFIXATIONMAPS.zip' , "r") as zip_ref:
    for file in zip_ref.namelist():
        file_name = os.path.split(file)[1]

        if file.endswith("Map.jpg"):
            file_path = "/content/mit/saliencymap/" + file_name

            with open(file_path, "wb") as saliency:
                saliency.write(zip_ref.read(file))


os.remove("./ALLFIXATIONMAPS.zip")
print("done!", flush=True)


--2020-12-07 06:52:34--  http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLSTIMULI.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235000497 (224M) [application/zip]
Saving to: ‘ALLSTIMULI.zip’

ALLSTIMULI.zip      100%[===================>] 224.11M  8.27MB/s    in 49s     

2020-12-07 06:53:23 (4.57 MB/s) - ‘ALLSTIMULI.zip’ saved [235000497/235000497]

--2020-12-07 06:53:23--  http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLFIXATIONMAPS.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9665209 (9.2M) [application/zip]
Saving to: ‘ALLFIXATIONMAPS.zip’

ALLFIXATIONMAPS.zip 100%[===================>]   9.22M  7.17MB/s    in 1.3s    

2020-12-07 06:

In [ ]:

from pkg_resources import resource_string
from scipy.io import loadmat

def build_padded_2d_array(arrays, max_length=None, padding_value=np.nan):
    if max_length is None:
        max_length = np.max([len(a) for a in arrays])

    output = np.ones((len(arrays), max_length), dtype=np.asarray(arrays[0]).dtype)
    output *= padding_value
    for i, array in enumerate(arrays):
        output[i, :len(array)] = array

    return output


!wget http://people.csail.mit.edu/tjudd/WherePeopleLook/DATA.zip
!wget http://people.csail.mit.edu/tjudd/WherePeopleLook/Code/DatabaseCode.zip 

temp_dir = './'

print('Creating fixations')
f = zipfile.ZipFile(os.path.join(temp_dir, 'DATA.zip'))
namelist = f.namelist()
namelist = filter_files(namelist, ['.svn', '__MACOSX', '.DS_Store'])
f.extractall(temp_dir , namelist)
first_fixation = 1
f = zipfile.ZipFile(os.path.join(temp_dir, 'DatabaseCode.zip'))
namelist = f.namelist()
namelist = filter_files(namelist, ['.svn', '__MACOSX', '.DS_Store'])
f.extractall(temp_dir, namelist)

subjects = glob.glob(os.path.join(temp_dir, 'DATA', '*'))
# Exclude files
subjects = [s for s in subjects if not os.path.splitext(s)[1]]
subjects = [os.path.basename(s) for s in subjects]
subjects = sorted(subjects)

os.remove("./DATA.zip")
os.remove("./DatabaseCode.zip")

with open(os.path.join(temp_dir, 'extract_fixations.m'), 'wb') as f:
    f.write(resource_string(__name__, 'scripts/{}'.format('extract_fixations.m')))

cmds = []
# It is vital _not_ to store the extracted fixations in the main
# directory where matlab is running, as matlab will check the timestamp
# of all files in this directory very often. This leads to heavy
# performance penalties and would make matlab run for more than an
# hour.
out_path = 'extracted'
os.makedirs(os.path.join(temp_dir, out_path))
total_cmd_count = len(mit_stimuli_filenames) * len(subjects)
for n, stimulus in enumerate(mit_stimuli_filenames):
    for subject_id, subject in enumerate(subjects):
        subject_path = os.path.join('DATA', subject)
        outfile = '{0}_{1}.mat'.format(stimulus, subject)
        outfile = os.path.join(out_path, outfile)
        cmds.append("fprintf('%d/%d\\n', {}, {});".format(n * len(subjects) + subject_id, total_cmd_count))
        cmds.append("extract_fixations('{0}', '{1}', '{2}');".format(stimulus, subject_path, outfile))

print('Running original code to extract fixations. This can take some minutes.')
print('Warning: In the IPython Notebook, the output is shown on the console instead of the notebook.')
with open(os.path.join(temp_dir, 'extract_all_fixations.m'), 'w') as f:
    for cmd in cmds:
        f.write('{}\n'.format(cmd))


--2020-12-07 06:53:26--  http://people.csail.mit.edu/tjudd/WherePeopleLook/DATA.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47024024 (45M) [application/zip]
Saving to: ‘DATA.zip’

DATA.zip            100%[===================>]  44.84M  2.23MB/s    in 16s     

2020-12-07 06:53:42 (2.85 MB/s) - ‘DATA.zip’ saved [47024024/47024024]

--2020-12-07 06:53:42--  http://people.csail.mit.edu/tjudd/WherePeopleLook/Code/DatabaseCode.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16690 (16K) [application/zip]
Saving to: ‘DatabaseCode.zip’

DatabaseCode.zip    100%[===================>]  16.30K  --.-KB/s    in 0.01s   

2020-12-07 06:53:42 (1.31 MB/s) - ‘Datab

In [ ]:
!apt install octave
!apt install liboctave-dev 
!apt-get install octave-statistics


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aglfn epstool fonts-droid-fallback fonts-noto-mono ghostscript gnuplot-data
  gnuplot-qt gsfonts imagemagick-6-common info install-info libamd2
  libauthen-sasl-perl libcamd2 libccolamd2 libcholmod3 libcolamd2
  libcupsfilters1 libcupsimage2 libcxsparse3 libdata-dump-perl libemf1
  libencode-locale-perl libfftw3-single3 libfile-listing-perl libfltk-gl1.3
  libfltk1.3 libfont-afm-perl libgail-common libgail18 libglpk40
  libgraphicsmagick++-q16-12 libgraphicsmagick-q16-3 libgs9 libgs9-common
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libhtml-form-perl
  libhtml-format-perl libhtml-parser-perl libhtml-tagset-perl
  libhtml-tree-perl libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libijs-0.35 libio-html-perl
  libio-socket-ssl-perl libjbig2dec0 liblqr-1-0 liblua5.3-0
  liblwp-mediat

In [ ]:
!octave

# type below line in the octave prompt to load the package:

# pkg load statistics

#update packages using below code

# pkg update

#reload the statistics package

# pkg load statistics

# To see a list of installed packages you can type

#pkg list

# Then type below code in the octave prompt:

# run("./extract_all_fixations.m")

#After execution of the code was over, close the prompt by stopping the execution of the cell
#To stop execution click twice on the square icon on the top left corner of the cell

Streaming output truncated to the last 5000 lines.
Loading DATA/CNG istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14626/15045
Loading DATA/ajs istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14627/15045
Loading DATA/emb istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14628/15045
Loading DATA/ems istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14629/15045
Loading DATA/ff istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14630/15045
Loading DATA/hp istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14631/15045
Loading DATA/jcw istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14632/15045
Loading DATA/jw istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14633/15045
Loading DATA/kae istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14634/15045
Loading DATA/krl istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14635/15045
Loading DATA/po istatic_outdoor_street_city_cambridge_uk_IMG_8702.jpeg
14636/15045
Loading DATA/tmj istatic_outd

In [ ]:

xs = []
ys = []
ts = []
ns = []
train_subjects = []
duration_hist = []
train_durations = []
for n, stimulus in enumerate(mit_stimuli_filenames):
    stimulus_size = mit_stimuli.sizes[n]
    for subject_id, subject in enumerate(subjects):
        subject_name = os.path.split(subject)[-1]
        outfile = '{0}_{1}.mat'.format(stimulus, subject_name)
        mat_data = loadmat(os.path.join(temp_dir, out_path, outfile))
        fix_data = mat_data['fixations']
        starts = mat_data['starts']
        _durations = mat_data['durations']
        #print(fix_data)
        x = []
        y = []
        t = []
        duration = []
        # if first_fixation == 1 the first fixation is skipped, as done
        # by Judd.
        # TODO: This contains a subtle inconsistency: if the first fixation is invalid,
        # the next fixation is _not_ skipped. Therefore, the dataset still
        # contains a few "real" initial fixations. However, this is consistent
        # with how the dataset has been used in the past, so we are
        # keeping it.
        for i in range(first_fixation, fix_data.shape[0]):
            if fix_data[i, 0] < 0 or fix_data[i, 1] < 0:
                continue
            if fix_data[i, 0] >= stimulus_size[1] or fix_data[i, 1] >= stimulus_size[0]:
                continue
            x.append(fix_data[i, 0])
            y.append(fix_data[i, 1])
            t.append(starts[0, i] / 240.0)  # Eye Tracker rate = 240Hz
            duration_hist.append(np.array(duration))
            #print(duration_hist)
            duration.append(_durations[0, i] / 1000)  # data is in ms, we want seconds
        xs.append(x)
        ys.append(y)
        ts.append(t)
        ns.append(n)
        train_subjects.append(subject_id)
        train_durations.append(duration)

#print(duration_hist)
attributes = {
    # duration_hist contains for each fixation the durations of the previous fixations in the scanpath
    'duration_hist': build_padded_2d_array(duration_hist),
}
scanpath_attributes = {
    # train_durations contains the fixation durations for each scanpath
    'train_durations': build_padded_2d_array(train_durations),
}
mit_fixations = FixationTrains.from_fixation_trains(xs, ys, ts, ns, train_subjects, attributes=attributes, scanpath_attributes=scanpath_attributes)




In [ ]:
!rm -r "./ALLSTIMULI"
!rm -r "./DATA"

**Download SALICON Dataset for training the readout network**

In [ ]:
#SALICON Dataset for training the readout network
from tqdm import tqdm
from scipy.io import loadmat

!gdown --id "1g8j-hTT-51IG1UFwP0xTGhLdgIUCW5e5"
!gdown --id "1PnO7szbdub1559LfjYHMy65EDC4VhJC8"
!gdown --id "1P-jeZXCsjoKO79OhFUgnj6FGcyvmLDPj"

!mkdir "/content/salicon"
!mkdir "/content/salicon/saliencymap/"
!mkdir "/content/salicon/fixations/"

!unzip -q maps.zip -d './salicon/saliencymap'


temp_dir = './'
print("Extracting stimuli")

f = zipfile.ZipFile('./images.zip')
f.extractall(temp_dir)


salicon_train_stimuli_name = [os.path.basename(f) for f in sorted(glob.glob(os.path.join(temp_dir, 'images', 'train', 'COCO_train*')))]
salicon_stimuli_train = create_stimuli(
    stimuli_location=os.path.join(temp_dir, 'images', 'train'),
    filenames= salicon_train_stimuli_name,
    location = './salicon/stimuli/train')


salicon_val_stimuli_name = [os.path.basename(f) for f in sorted(glob.glob(os.path.join(temp_dir, 'images', 'val', 'COCO_val*')))]
salicon_stimuli_val = create_stimuli(
    stimuli_location=os.path.join(temp_dir, 'images', 'val'),
    filenames= salicon_val_stimuli_name,
    location = './salicon/stimuli/val')


salicon_test_stimuli_name = [os.path.basename(f) for f in sorted(glob.glob(os.path.join(temp_dir, 'images', 'test', 'COCO_test*')))]
salicon_stimuli_test = create_stimuli(
    stimuli_location=os.path.join(temp_dir, 'images', 'test'),
    filenames= salicon_test_stimuli_name,
    location = './salicon/stimuli/test')


f = zipfile.ZipFile('./fixations.zip')
f.extractall('/content/salicon/fixations/')

fixations = []

fixation_attr = 'fixations'
fixation_type =  'fixations'
for dataset in ['train', 'val']:
    ns = []
    train_xs = []
    train_ys = []
    train_ts = []
    train_subjects = []

    subject_id = 0

    data_files = list(sorted(glob.glob(os.path.join('/content/salicon/fixations', dataset, '*.mat'))))
    for n, filename in enumerate(tqdm(data_files)):
        fixation_data = loadmat(filename)
        for subject_data in fixation_data['gaze'].flatten():
            train_xs.append(subject_data[fixation_attr][:, 0] - 1)  # matlab: one-based indexing
            train_ys.append(subject_data[fixation_attr][:, 1] - 1)
            if fixation_type == 'mouse':
                train_ts.append(subject_data['timestamp'].flatten())
            elif fixation_type == 'fixations':
                train_ts.append(range(len(train_xs[-1])))

            train_subjects.append(np.ones(len(train_xs[-1]), dtype=int) * subject_id)
            ns.append(np.ones(len(train_xs[-1]), dtype=int) * n)
            subject_id += 1

    xs = np.hstack(train_xs)
    ys = np.hstack(train_ys)
    ts = np.hstack(train_ts)
    subjects = np.hstack(train_subjects)
    ns = np.hstack(ns)

    fixations.append(Fixations.FixationsWithoutHistory(xs, ys, ts, ns, subjects))

salicon_fixations_train, salicon_fixations_val = fixations


import os 
os.remove("./images.zip")
os.remove("./maps.zip")
os.remove("./fixations.zip")

Downloading...
From: https://drive.google.com/uc?id=1g8j-hTT-51IG1UFwP0xTGhLdgIUCW5e5
To: /content/images.zip
2.27GB [00:27, 81.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PnO7szbdub1559LfjYHMy65EDC4VhJC8
To: /content/maps.zip
452MB [00:06, 72.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1P-jeZXCsjoKO79OhFUgnj6FGcyvmLDPj
To: /content/fixations.zip
1.54GB [00:33, 46.0MB/s]
Extracting stimuli


100%|██████████| 5000/5000 [00:15<00:00, 317.63it/s]


**The weights of pretrained VGG19 network should be normalized such that all filters be rescaled to yield feature maps with unit mean over the ImageNet dataset.**

**The normalized VGG19 network is provided as a caffe model by authors research center. We should convert caffe model to tensorflow model weights to load it on our architecture.**

In [ ]:
!git clone https://github.com/black-puppydog/vgg19-normalized-to-tensorflow.git 


In [ ]:
#Weights of vgg19 network trained on imagenet dataset
!wget http://bethgelab.org/media/uploads/deeptextures/vgg_normalised.caffemodel -P vgg19-normalized-to-tensorflow

In [ ]:
import os

!pip install Protobuf
!pip install PyCaffe
!apt install caffe-cuda

**Extract features from caffe model:**

In [ ]:
!pwd
os.chdir('/content/vgg19-normalized-to-tensorflow')
#! /usr/bin/env python3

# Copyright 2017 Daan Wynen
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import os
os.environ['GLOG_minloglevel'] = '2'
import caffe
import hashlib
import numpy as np
from matplotlib.pyplot import imread
import json

PROTOTXT_FNAME = 'VGG_ILSVRC_19_layers_deploy_fullconv.prototxt'
CAFFEMODEL_FNAME = 'vgg_normalised.caffemodel'


CAFFE_FEATURES_FNAME = 'features_caffe.npz'
CAFFE_FILTERS_FNAME = 'filters_caffe.npz'
IMAGE_CAT = 'cat.jpg'

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

expected_hash_prototxt = 'dc965efaff95dc64360c84ad4bce13d03f6cc062f9e8d2e40428e0c407360fe55f6a91d54980ea70ac6fdd6ccd724685e2a74fb145bf7e8fc0c954497a85357b'
expected_hash_caffemodel = '63f1af8188923a09fc0fd9c111ae7085e0a7045e9ddf2b6f97164e995034b65dbcebac4360bde154e6e771b16c4d6a32a4986fd809696d6a9d10f530271f6e15'

print('checking prototxt hash')
with open(PROTOTXT_FNAME, 'rb') as f:
  hash = hashlib.sha512(b''.join(f.readlines())).hexdigest()
  print(hash)
  #assert hash == expected_hash_prototxt
print('checking caffemodel hash')
with open(CAFFEMODEL_FNAME, 'rb') as f:
  hash = hashlib.sha512(b''.join(f.readlines())).hexdigest()
  #assert hash == expected_hash_caffemodel
print('ready to go')

image = imread(IMAGE_CAT)
net = caffe.Net(PROTOTXT_FNAME, CAFFEMODEL_FNAME,
                caffe.TEST)

# remove mean channels, move channels to outermost dimension, and insert dimension for batch size 1
transformed_image = (image - [_R_MEAN, _G_MEAN, _B_MEAN]).transpose(2, 0, 1)[np.newaxis, ::-1, :, :]

net.blobs['data'].data[...] = transformed_image
blob_names = list(net.blobs.keys())
results = net.forward_all(blob_names, data=transformed_image)


np.savez(CAFFE_FEATURES_FNAME, **results)
weights = dict()
for k, v in net.params.items():
  print(k)
  print('{}\tbiases: {}\tweights:{}'.format(k, v[1].data.shape, v[0].data.shape))
  weights[k+'/weights'] = v[0].data
  weights[k+'/biases'] = v[1].data
  assert weights[k+'/weights'].ndim == 4
  assert weights[k+'/biases'].ndim == 1

np.savez(CAFFE_FILTERS_FNAME, **weights)

/content
checking prototxt hash
dd3b5a2bfd6227f5725c1520050eccaaeab4e4e62f45ffc23eccc3a141a88b6c837bc822fba8d323b589a6745c93547cd4aa3231bb47d2315a1bdde122d3a48c
checking caffemodel hash
ready to go
conv1_1
conv1_1	biases: (64,)	weights:(64, 3, 3, 3)
conv1_2
conv1_2	biases: (64,)	weights:(64, 64, 3, 3)
conv2_1
conv2_1	biases: (128,)	weights:(128, 64, 3, 3)
conv2_2
conv2_2	biases: (128,)	weights:(128, 128, 3, 3)
conv3_1
conv3_1	biases: (256,)	weights:(256, 128, 3, 3)
conv3_2
conv3_2	biases: (256,)	weights:(256, 256, 3, 3)
conv3_3
conv3_3	biases: (256,)	weights:(256, 256, 3, 3)
conv3_4
conv3_4	biases: (256,)	weights:(256, 256, 3, 3)
conv4_1
conv4_1	biases: (512,)	weights:(512, 256, 3, 3)
conv4_2
conv4_2	biases: (512,)	weights:(512, 512, 3, 3)
conv4_3
conv4_3	biases: (512,)	weights:(512, 512, 3, 3)
conv4_4
conv4_4	biases: (512,)	weights:(512, 512, 3, 3)
conv5_1
conv5_1	biases: (512,)	weights:(512, 512, 3, 3)
conv5_2
conv5_2	biases: (512,)	weights:(512, 512, 3, 3)
conv5_3
conv5_3	biases: (5

**Convert Caffe checkpoints to tensorflow:**

In [ ]:
#! /usr/bin/env python3

# Copyright 2017 Daan Wynen
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
#import vgg
from matplotlib.pyplot import imread
from scipy.stats import describe
import tensorflow as tf
from tensorflow.contrib.slim.nets import vgg
slim = tf.contrib.slim
!mkdir '../weights/'
CHECKPOINT_PATH = '../weights/vgg19_normalized'

CAFFE_FEATURES_FNAME = 'features_caffe.npz'
CAFFE_FILTERS_FNAME = 'filters_caffe.npz'
IMAGE_CAT = 'cat.jpg'

# set to true to dump the two .npz files for further inspection
SAVE_OUTPUTS = False
TF_FEATURES_FNAME = 'features_tf.npz'
CAFFE_FEATURES_TF_FNAME = 'features_caffe_tforder.npz'

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

npz = np.load(CAFFE_FEATURES_FNAME)

image_rgb = tf.constant(imread(IMAGE_CAT)[np.newaxis], dtype=tf.float32)
image_rgb_centered = tf.subtract(image_rgb, [_R_MEAN, _G_MEAN, _B_MEAN], 'image_rgb_centered')


# build the tf graph we want to load the weigts and biases into
with slim.arg_scope(vgg.vgg_arg_scope()):
  _, end_points = vgg.vgg_19(image_rgb_centered,
                             is_training=False,
                             spatial_squeeze=False)
  

# get the variables for the convolutions.
# the caffemodel does not contain the fully-connected layers,
# so we ignore these here
vgg_conv_variables = {v.name: v for v in slim.get_model_variables('vgg_19') if '/fc' not in v.name}

print('reading caffe filters')
caffe_filters = np.load(CAFFE_FILTERS_FNAME)
assignments = dict()
for k, v in sorted(caffe_filters.items(), key=lambda p: p[0]):
  block = k[:5]
  target_var = vgg_conv_variables['vgg_19/{}/{}:0'.format(block, k)]
  if target_var.name.endswith('biases:0'):
    assignments[target_var.name] = v
  else:
    assert target_var.name.endswith('weights:0')
    assignments[target_var.name] = v.transpose(2, 3, 1, 0)

# invert the pixel order for the first convolution to go from BGR to RGB inputs
assignments['vgg_19/conv1/conv1_1/weights:0'] = assignments['vgg_19/conv1/conv1_1/weights:0'][:, :, ::-1, :]

assign_op, assign_feed = slim.assign_from_values(assignments)

print('reading caffe activations')
caffe_npz = np.load(CAFFE_FEATURES_FNAME)

activation_targets = dict()
caffe_activations = dict()
for k, v in sorted(caffe_npz.items(), key=lambda p: p[0]):
  if k.startswith('pool'):
    t_name = 'vgg_19/'+k
    caffe_activations[t_name] = v.transpose(0, 2, 3, 1)
    activation_targets[t_name] = end_points[t_name]
  elif k == 'data':
    continue
  else:
    block = k[:5]
    ep_name = 'vgg_19/{}/{}'.format(block, k)
    caffe_activations[ep_name] = v.transpose(0, 2, 3, 1)
    activation_targets[ep_name] = end_points[ep_name]


with tf.Session() as sess:
  print('initializing')
  sess.run(tf.global_variables_initializer())

  print('assigning caffe weights')
  sess.run(assign_op, feed_dict=assign_feed)

  print('saving checkpoint to {}'.format(CHECKPOINT_PATH))
  # we only save the variables we're after
  saver = tf.train.Saver(list(vgg_conv_variables.values()))
  saver.save(sess, save_path=CHECKPOINT_PATH)

  tf_activations = sess.run(activation_targets)

if SAVE_OUTPUTS:
  np.savez(TF_FEATURES_FNAME, **activations)
  np.savez(CAFFE_FEATURES_TF_FNAME, **caffe_activations)

# print some stats on how well the activations at different layers match
for k in sorted(caffe_activations.keys()):
  print(k)
  ck = caffe_activations[k]
  tk = tf_activations[k]
  diff = np.abs((ck - tk))
  print(describe(diff.flatten()))
  print(np.percentile(diff, 99))
  print()
  
print('DONE')
os.chdir('/content/')

**Determining the hyper-parameters:**

In [ ]:
class config:
  """General training parameters that define the maximum number of
    training epochs, the batch size, and learning rate for the ADAM
    optimization method. The device should be
    "gpu".
  """

  PARAMS = {
      "n_epochs": 25,
      "batch_size": 1,
      "learning_rate": 1e-5,
      "device": "gpu"
  }

  """The predefined input image sizes for each of the 2 datasets.
     as suggested by the authors mit 1003 images are converted to 1024 × 768 or 768 × 1024
     depending on the aspect ratio and then subsampled by a factor of two. Therefore 
     they are either 384x512 or 512x384 before entering the network.
  """

  DIMS = {
      "image_size_salicon": (240, 320),
      "image_size_mit1003_1": (384, 512),
      "image_size_mit1003_2": (512, 384),
  }

**Preprocessig the data:**

In [ ]:
import os
import sys

import numpy as np
import tensorflow as tf

class SALICON:
    """This class represents the SALICON dataset. It consists of 10000 training
       and 5000 validation images. The corresponding mouse tracking maps were
       acquired from 16 observers during free-viewing of images for 5 seconds.
       All stimuli are of size 480x640 pixels (height by width).
    Attributes:
        n_train: Number of training instances as defined in the dataset.
        n_valid: Number of validation instances as defined in the dataset.
    Returns:
        tuple: A tuple that consists of dataset objects holding the training
               and validation set instances respectively.
    """

    n_train = 10000
    n_valid = 1003

    def __init__(self, data_path):


        self._target_size = config.DIMS["image_size_salicon"]

        ##The training set is Salicon 
        self._dir_stimuli_train = data_path + "salicon/stimuli/train"
        #self._dir_stimuli_valid = data_path + "salicon/stimuli/images/val"

        self._dir_saliency_train = data_path + "salicon/saliencymap/train"
        #self._dir_saliency_valid = data_path + "salicon/saliencymap/val"


        ##The validation set is MIT1003
        self._valid_target_size = config.DIMS["image_size_mit1003_1"]

        self._dir_stimuli_valid = data_path + "mit/stimuli"
        self._dir_saliency_valid = data_path + "mit/saliencymap"

    def load_data(self):
        train_list_x = _get_file_list(self._dir_stimuli_train)
        train_list_y = _get_file_list(self._dir_saliency_train)


        #train_indices = np.arange(10000)
        
        salicon_fixations_array_x = []
        salicon_fixations_array_y = []

        '''for i in range(10000):
          salicon_fixations_array_x.append([salicon_fixations_train[salicon_fixations_train.n==i].x])
          salicon_fixations_array_y.append([salicon_fixations_train[salicon_fixations_train.n==i].y])'''

  
        _check_consistency(zip(train_list_x, train_list_y), 10000)

        train_set = _fetch_dataset((train_list_x, train_list_y),#salicon_fixations_array_x, salicon_fixations_array_y),
                                   self._target_size, True)
        print(train_set)
        valid_list_x = _get_file_list(self._dir_stimuli_valid)
        valid_list_y = _get_file_list(self._dir_saliency_valid)
        
        #valid_indices = np.arange(1003)

        mit_fixations_array_x = []
        mit_fixations_array_y = []

        '''for i in range(1003):
          mit_fixations_array_x.append([mit_fixations[mit_fixations.n==i].x])
          mit_fixations_array_y.append([mit_fixations[mit_fixations.n==i].y]) '''

        _check_consistency(zip(valid_list_x, valid_list_y), 1003)

        valid_set = _fetch_dataset((valid_list_x, valid_list_y), #mit_fixations_array_x, mit_fixations_array_y),
                                   self._valid_target_size, False)

        return (train_set, valid_set)

class MIT1003:
    """This class represents the MIT1003 dataset. It consists of 1003 natural
       images. The corresponding fixation maps were acquired from 15 observers
       during free-viewing of images for 3 seconds. The largest dimension of
       all stimuli is of size 1024 pixels.
    Attributes:
        n_train: Assigned number of training instances (must sum to 1003).
        n_valid: Assigned number of validation instances (must sum to 1003).
    Returns:
        tuple: A tuple that consists of dataset objects holding the training
               and validation set instances respectively.
    """

    n_train = 803
    n_valid = 200

    def __init__(self, data_path):
        self._target_size = config.DIMS["image_size_mit1003_1"]

        self._dir_stimuli = data_path + "mit/stimuli"
        self._dir_saliency = data_path + "mit/saliencymap"

    def load_data(self):

        list_x = _get_file_list(self._dir_stimuli)
        list_y = _get_file_list(self._dir_saliency)

        _check_consistency(zip(list_x, list_y), 1003)

        indices = _get_random_indices(1003)
        excerpt = indices[:self.n_train]

        train_list_x = [list_x[idx] for idx in excerpt]
        train_list_y = [list_y[idx] for idx in excerpt]

        train_set = _fetch_dataset((train_list_x, train_list_y),
                                   self._target_size, True)

        excerpt = indices[self.n_train:]

        valid_list_x = [list_x[idx] for idx in excerpt]
        valid_list_y = [list_y[idx] for idx in excerpt]

        valid_set = _fetch_dataset((valid_list_x, valid_list_y),
                                   self._target_size, False)

        return (train_set, valid_set)


class TEST:
    """This class represents test set instances used for inference through
       a trained network. All stimuli are resized to the preferred spatial
       dimensions of the chosen model. This can, however, lead to cases of
       excessive image padding.
    Returns:
        object: A dataset object that holds all test set instances
                specified under the path variable.
    """

    def __init__(self, dataset, data_path):
        self._target_size = config.DIMS["image_size_mit1003_1"]

        self._dir_stimuli_test = data_path+ "mit/stimuli"

    def load_data(self):
        test_list_x = _get_file_list(self._dir_stimuli_test)

        test_set = _fetch_dataset(test_list_x, self._target_size,
                                  False, online=True)
        return test_set


def get_dataset_iterator(phase, dataset, data_path):
    """Entry point to make an initializable dataset iterator for either
       training or testing a model by calling the respective dataset class.
    Args:
        phase (str): Holds the current phase, which can be "train" or "test".
        dataset (str): Denotes the dataset to be used during training or the
                       suitable resizing procedure when testing a model.
        data_path (str): Points to the directory where training or testing
                         data instances are stored.
    Returns:
        iterator: An initializable dataset iterator holding the relevant data.
        initializer: An operation required to initialize the correct iterator.
    """

    if phase == "train":
        current_module = sys.modules[__name__]
        class_name = "%s" % dataset.upper()
        class_name = eval(class_name)
        dataset_class = class_name(data_path)
        train_set, valid_set = dataset_class.load_data()

        iterator = tf.data.Iterator.from_structure(train_set.output_types,
                                                   train_set.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_set)
        valid_init_op = iterator.make_initializer(valid_set)

        return next_element, train_init_op, valid_init_op

    if phase == "fine-tune":
        current_module = sys.modules[__name__]
        class_name = "%s" % dataset.upper()
        class_name = eval(class_name)
        dataset_class = class_name(data_path)
        train_set, valid_set = dataset_class.load_data()

        iterator = tf.data.Iterator.from_structure(train_set.output_types,
                                                   train_set.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_set)
        valid_init_op = iterator.make_initializer(valid_set)

        return next_element, train_init_op, valid_init_op

    if phase == "test":
        test_class = TEST(dataset, data_path)
        test_set = test_class.load_data()

        iterator = tf.data.Iterator.from_structure(test_set.output_types,
                                                   test_set.output_shapes)
        next_element = iterator.get_next()

        init_op = iterator.make_initializer(test_set)

        return next_element, init_op


def postprocess_saliency_map(saliency_map, target_size):
    """This function resizes and crops a single saliency map to the original
       dimensions of the input image. The output is then encoded as a jpeg
       file suitable for saving to disk.
    Args:
        saliency_map (tensor, float32): 3D tensor that holds the values of a
                                        saliency map in the range from 0 to 1.
        target_size (tensor, int32): 1D tensor that specifies the size to which
                                     the saliency map is resized and cropped.
    Returns:
        tensor, str: A tensor of the saliency map encoded as a jpeg file.
    """


    saliency_map *= 255.0

    saliency_map = _resize_image(saliency_map, target_size, True)
    saliency_map = _crop_image(saliency_map, target_size)

    saliency_map = tf.round(saliency_map)
    saliency_map = tf.cast(saliency_map, tf.uint8)

    saliency_map_jpeg = tf.image.encode_jpeg(saliency_map, "grayscale", 100)

    return saliency_map_jpeg


def _fetch_dataset(files, target_size, shuffle, online=False):
    
    """
    Here the list of file directories is shuffled (only when training),
       loaded, batched, and prefetched to ensure high GPU utilization.
    Args:
        files (list, str): A list that holds the paths to all file instances.
        target_size (tuple, int): A tuple that specifies the size to which
                                  the data will be reshaped.
        shuffle (bool): Determines whether the dataset will be shuffled or not.
        online (bool, optional): Flag that decides whether the batch size must
                                 be 1 or can take any value. Defaults to False.
    Returns:
        object: A dataset object that contains the batched and prefetched data
                instances along with their shapes and file paths.
    """


    dataset = tf.data.Dataset.from_tensor_slices(files)

    if shuffle:
        dataset = dataset.shuffle(len(files[0]))

    
    dataset = dataset.map(lambda *files: _parse_function(files, target_size),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    batch_size = 1 if online else config.PARAMS["batch_size"]

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(5)

    return dataset


def _parse_function(files, target_size):
    """This function reads image data dependent on the image type and
       whether it constitutes a stimulus or saliency map. All instances
       are then reshaped and padded to yield the target dimensionality.
    Args:
        files (tuple, str): A tuple with the paths to all file instances.
                            The first element contains the stimuli and, if
                            present, the second one the ground truth maps.
        target_size (tuple, int): A tuple that specifies the size to which
                                  the data will be reshaped.
    Returns:
        list: A list that holds the image instances along with their
              shapes and file paths.
    """

    image_list = []

    #files_1 , files_2 , f_x , f_y = files
    #print(tf.shape(files_2))

    for count, filename in enumerate(files):
        #print(count)
        image_str = tf.read_file(filename)
        channels = 3 if count == 0 else 1

        image = tf.cond(tf.image.is_jpeg(image_str),
                        lambda: tf.image.decode_jpeg(image_str,
                                                     channels=channels),
                        lambda: tf.image.decode_png(image_str,
                                                    channels=channels))
        original_size = tf.shape(image)[:2]

        target_size = tf.cond(tf.math.logical_and(target_size==["image_size_mit1003_1"] , original_size[0]> original_size[1]), lambda:config.DIMS["image_size_mit1003_2"], false_fn=lambda:config.DIMS["image_size_mit1003_1"])

        image = _resize_image(image, target_size)
        image = _pad_image(image, target_size)

        image_list.append(image)
        
        '''if count%2==1:
          image_list.append(np.asarray(f_x)[count])
          image_list.append(np.asarray(f_y)[count])'''

    image_list.append(original_size)
    image_list.append(files)
    return image_list


def _resize_image(image, target_size, overfull=False):
    """This resizing procedure preserves the original aspect ratio and might be
       followed by padding or cropping. Depending on whether the target size is
       smaller or larger than the current image size, the area or bicubic
       interpolation method will be utilized.
    Args:
        image (tensor, uint8): A tensor with the values of an image instance.
        target_size (tuple, int): A tuple that specifies the size to which
                                  the data will be resized.
        overfull (bool, optional): Denotes whether the resulting image will be
                                   larger or equal to the specified target
                                   size. This is crucial for the following
                                   padding or cropping. Defaults to False.
    Returns:
        tensor, float32: 4D tensor that holds the values of the resized image.
    """

    current_size = tf.shape(image)[:2]

    target_size = tf.cond(tf.math.logical_and(target_size==["image_size_mit1003_1"] , current_size[0]> current_size[1]), lambda:config.DIMS["image_size_mit1003_2"], false_fn=lambda:config.DIMS["image_size_mit1003_1"])


    height_ratio = target_size[0] / current_size[0]
    width_ratio = target_size[1] / current_size[1]

    if overfull:
        target_ratio = tf.maximum(height_ratio, width_ratio)
    else:
        target_ratio = tf.minimum(height_ratio, width_ratio)

    target_size = tf.cast(current_size, tf.float64) * target_ratio
    target_size = tf.cast(tf.round(target_size), tf.int32)

    shrinking = tf.cond(tf.logical_or(current_size[0] > target_size[0],
                                      current_size[1] > target_size[1]),
                        lambda: tf.constant(True),
                        lambda: tf.constant(False))

    image = tf.expand_dims(image, 0)

    image = tf.cond(shrinking,
                    lambda: tf.image.resize_area(image, target_size,
                                                 align_corners=True),
                    lambda: tf.image.resize_bicubic(image, target_size,
                                                    align_corners=True))

    image = tf.clip_by_value(image[0], 0.0, 255.0)

    return image


def _pad_image(image, target_size):
    """A single image, either stimulus or saliency map, will be padded
       symmetrically with the constant value 126 or 0 respectively.
    Args:
        image (tensor, float32): 3D tensor with the values of the image data.
        target_size (tuple, int): A tuple that specifies the size to which
                                  the data will be resized.
    Returns:
        tensor, float32: 3D tensor that holds the values of the padded image.
    """
    current_size = tf.shape(image)[:2]

    target_size = tf.cond(tf.math.logical_and(target_size==["image_size_mit1003_1"] , current_size[0]> current_size[1]), lambda:config.DIMS["image_size_mit1003_2"], false_fn=lambda:config.DIMS["image_size_mit1003_1"])


    current_size = tf.shape(image)

    pad_constant_value = tf.cond(tf.equal(current_size[2], 3),
                                 lambda: tf.constant(126.0),
                                 lambda: tf.constant(0.0))

    pad_vertical = (target_size[0] - current_size[0]) / 2
    pad_horizontal = (target_size[1] - current_size[1]) / 2

    pad_top = tf.floor(pad_vertical)
    pad_bottom = tf.ceil(pad_vertical)
    pad_left = tf.floor(pad_horizontal)
    pad_right = tf.ceil(pad_horizontal)

    padding = [[pad_top, pad_bottom], [pad_left, pad_right], [0, 0]]
    image = tf.pad(image, padding, constant_values=pad_constant_value)

    return image

def _crop_image(image, target_size):
    """A single saliency map will be cropped according the specified target
       size by extracting the central region of the image and correctly
       removing the added padding.
    Args:
        image (tensor, float32): 3D tensor with the values of a saliency map.
        target_size (tensor, int32): 2D tensor that specifies the size to
                                     which the data will be cropped.
    Returns:
        tensor, float32: 3D tensor that holds the values of the saliency map
                         with cropped dimensionality.
    """
    current_size = tf.shape(image)[:2]

    target_size = tf.cond(tf.math.logical_and(target_size==["image_size_mit1003_1"] , current_size[0]> current_size[1]), lambda:config.DIMS["image_size_mit1003_2"], false_fn=lambda:config.DIMS["image_size_mit1003_1"])


    current_size = tf.shape(image)[:2]

    crop_vertical = (current_size[0] - target_size[0]) / 2
    crop_horizontal = (current_size[1] - target_size[1]) / 2

    crop_top = tf.cast(tf.floor(crop_vertical), tf.int32)
    crop_left = tf.cast(tf.floor(crop_horizontal), tf.int32)

    border_bottom = crop_top + target_size[0]
    border_right = crop_left + target_size[1]

    image = image[crop_top:border_bottom, crop_left:border_right, :]

    return image


def _get_file_list(data_path):
    """This function detects all image files within the specified parent
       directory for either training or testing. The path content cannot
       be empty, otherwise an error occurs.
    Args:
        data_path (str): Points to the directory where training or testing
                         data instances are stored.
    Returns:
        list, str: A sorted list that holds the paths to all file instances.
    """

    data_list = []

    if os.path.isfile(data_path):
        data_list.append(data_path)
    else:
        for subdir, dirs, files in os.walk(data_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    data_list.append(os.path.join(subdir, file))

    data_list.sort()

    if not data_list:
        raise FileNotFoundError("No data was found")

    return data_list


def _get_random_indices(list_length):
    """A helper function to generate an array of randomly shuffled indices
       to divide the MIT1003 and CAT2000 datasets into training and validation
       instances.
    Args:
        list_length (int): The number of indices that is randomly shuffled.
    Returns:
        array, int: A 1D array that contains the shuffled data indices.
    """

    indices = np.arange(list_length)
    prng = np.random.RandomState(42)
    prng.shuffle(indices)

    return indices


def _check_consistency(zipped_file_lists, n_total_files):
    """A consistency check that makes sure all files could successfully be
       found and stimuli names correspond to the ones of ground truth maps.
    Args:
        zipped_file_lists (tuple, str): A tuple of train and valid path names.
        n_total_files (int): The total number of files expected in the list.
    """

    assert len(list(zipped_file_lists)) == n_total_files, "Files are missing"

    for file_tuple in zipped_file_lists:
        file_names = [os.path.basename(entry) for entry in list(file_tuple)]
        file_names = [os.path.splitext(entry)[0] for entry in file_names]
        file_names = [entry.replace("_fixMap", "") for entry in file_names]
        file_names = [entry.replace("_fixPts", "") for entry in file_names]

        assert len(set(file_names)) == 1, "File name mismatch"


**Computing center bias density for the two datasets:**


In [ ]:
bandwidth = 0.0217
regularization = 2.0e-13
from .baseline_utils import BaselineModel , KDEGoldModel

MIT_baseline = BaselineModel(mit_stimuli, mit_fixations, bandwidth=bandwidth, eps=regularization)
Salicon_baseline = BaselineModel(salicon_stimuli_train, salicon_fixations_train, bandwidth=bandwidth, eps=regularization)

image = mit_stimuli[56]
MIT_log_baseline =MIT_baseline.log_density(image)
print('mit baseline' , MIT_log_baseline)
print('\n')

image = salicon_stimuli_train[18]
SAL_log_baseline =Salicon_baseline.log_density(image)
print('salicon baseline', SAL_log_baseline)


mit baseline [[-17.2150465  -17.21595815 -17.21778055 ... -23.59797709 -23.62804533
  -23.64335387]
 [-17.21709925 -17.21800935 -17.21982865 ... -23.58510213 -23.61500474
  -23.6302273 ]
 [-17.22112686 -17.22203391 -17.22384708 ... -23.56158707 -23.59118715
  -23.60625268]
 ...
 [-19.71457303 -19.70959545 -19.69971585 ... -18.88426369 -18.8980179
  -18.90511107]
 [-19.7322474  -19.72723213 -19.71727795 ... -18.88564576 -18.89942076
  -18.90652533]
 [-19.74124313 -19.73620871 -19.72621668 ... -18.8861566  -18.89993977
  -18.90704885]]


salicon baseline [[-21.34211636 -21.33242133 -21.31340594 ... -20.31739044 -20.32728824
  -20.33242482]
 [-21.30825948 -21.29861143 -21.27968232 ... -20.29628727 -20.30605881
  -20.31112774]
 [-21.24234827 -21.23280132 -21.2140596  ... -20.25576865 -20.26530772
  -20.27025205]
 ...
 [-21.42174826 -21.40984875 -21.38646556 ... -22.8793721  -22.91204999
  -22.92793775]
 [-21.45074953 -21.43863991 -21.41485988 ... -22.97505829 -23.00845831
  -23.02465336]
 

**Building the neural network architecture, and loading pretrained weights:**

In [ ]:
import os
from scipy.ndimage.filters import gaussian_filter
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

class DeepGaze2:
    """The class representing deepgaze2 based on the VGG19 model. It
       implements a definition of the computational graph, as well as
       functions related to network training.
    """

    def __init__(self):
        self._output = None
        self._mapping = {}

        if config.PARAMS["device"] == "gpu":
            self._data_format = "channels_first"
            self._channel_axis = 1
            self._dims_axis = (2, 3)
        elif config.PARAMS["device"] == "cpu":
            self._data_format = "channels_last"
            self._channel_axis = 3
            self._dims_axis = (1, 2)

    def _encoder(self, images):
        """The encoder of the model consists of a pretrained VGG16 architecture
           with 13 convolutional layers. All dense layers are discarded and the
           last 3 layers are dilated at a rate of 2 to account for the omitted
           downsampling. Finally, the activations from 3 layers are combined.
        Args:
            images (tensor, float32): A 4D tensor that holds the RGB image
                                      batches used as input to the network.
        """

        imagenet_mean = tf.constant([103.939, 116.779, 123.68])
        imagenet_mean = tf.reshape(imagenet_mean, [1, 1, 1, 3])

        images -= imagenet_mean

        if self._data_format == "channels_first":
            images = tf.transpose(images, (0, 3, 1, 2))

        layer01 = tf.layers.conv2d(images, 64, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv1/conv1_1")

        layer02 = tf.layers.conv2d(layer01, 64, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv1/conv1_2")

        layer03 = tf.layers.max_pooling2d(layer02, 2, 2,
                                          data_format=self._data_format)

        layer04 = tf.layers.conv2d(layer03, 128, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv2/conv2_1")

        layer05 = tf.layers.conv2d(layer04, 128, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv2/conv2_2")

        layer06 = tf.layers.max_pooling2d(layer05, 2, 2,
                                          data_format=self._data_format)

        layer07 = tf.layers.conv2d(layer06, 256, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv3/conv3_1")

        layer08 = tf.layers.conv2d(layer07, 256, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv3/conv3_2")

        layer09 = tf.layers.conv2d(layer08, 256, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv3/conv3_3")
        
        layer10 = tf.layers.conv2d(layer09, 256, 3,
                          padding="same",
                          activation=tf.nn.relu,
                          data_format=self._data_format,
                          name="vgg_19/conv3/conv3_4")

        layer11 = tf.layers.max_pooling2d(layer10, 2, 2,
                                          data_format=self._data_format)

        layer12 = tf.layers.conv2d(layer11, 512, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv4/conv4_1")

        layer13 = tf.layers.conv2d(layer12, 512, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv4/conv4_2")

        layer14 = tf.layers.conv2d(layer13, 512, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv4/conv4_3")
        
        layer15 = tf.layers.conv2d(layer14, 512, 3,
                                   padding="same",
                                   activation=tf.nn.relu,
                                   data_format=self._data_format,
                                   name="vgg_19/conv4/conv4_4")

        layer16 = tf.layers.max_pooling2d(layer15, 2, 2,
                                          padding="same",
                                          data_format=self._data_format)

        layer17 = tf.layers.conv2d(layer16, 512, 3,
                                   padding="same",
                                   activation=None,
                                   data_format=self._data_format,
                                   name="vgg_19/conv5/conv5_1")
        layer18 = tf.nn.relu(layer17 ,
                             name="vgg_19/conv5/relu5_1")

        layer19 = tf.layers.conv2d(layer18, 512, 3,
                                   padding="same",
                                   activation=None,
                                   data_format=self._data_format,
                                   name="vgg_19/conv5/conv5_2")
        layer20 = tf.nn.relu(layer19 ,
                             name="vgg_19/conv5/relu5_2")
        
        layer21 = tf.layers.conv2d(layer20, 512, 3,
                                   padding="same",
                                   activation=None,
                                   data_format=self._data_format,
                                   name="vgg_19/conv5/conv5_3")
        layer22 = tf.nn.relu(layer21 ,
                             name="vgg_19/conv5/relu5_3")
        
        layer23 = tf.layers.conv2d(layer22, 512, 3,
                                   padding="same",
                                   activation=None,
                                   data_format=self._data_format,
                                   name="vgg_19/conv5/conv5_4")
        layer24 = tf.nn.relu(layer23 ,
                             name="vgg_19/conv5/relu5_4")
        
        layer25 = tf.layers.max_pooling2d(layer24, 2, 2,
                                          padding="same",
                                          data_format=self._data_format)

        shape = tf.shape(layer17)

        up_layer17 = self._upsample(layer17, shape, 16)

        up_layer18 = self._upsample(layer18, shape, 16)

        up_layer20 = self._upsample(layer20, shape, 16)

        up_layer21 = self._upsample(layer21, shape, 16)

        up_layer24 = self._upsample(layer24, shape, 16)


        encoder_output = tf.concat([up_layer17, up_layer18, up_layer20, up_layer21, up_layer24],
                                   axis=self._channel_axis)

        self._output = encoder_output


    def _readout(self, features):
        """The decoder model applies a series of 3 upsampling blocks that each
           performs bilinear upsampling followed by a 3x3 convolution to avoid
           checkerboard artifacts in the image space. Unlike all other layers,
           the output of the model is not modified by a ReLU.
        Args:
            features (tensor, float32): A 4D tensor that holds the features
                                        produced by the vgg19 module.
        """

        layer1 = tf.layers.conv2d(features, 16, 1,
                                  padding="same",
                                  activation=tf.nn.relu,
                                  data_format=self._data_format,
                                  name="readout/conv1")

        layer2 = tf.layers.conv2d(layer1, 32, 1,
                                  padding="same",
                                  activation=tf.nn.relu,
                                  data_format=self._data_format,
                                  name="readout/conv2")

        layer3 = tf.layers.conv2d(layer2, 2, 1,
                                  padding="same",
                                  activation=tf.nn.relu,
                                  data_format=self._data_format,
                                  name="readout/conv3")

        readout_output = tf.layers.conv2d(layer3, 1, 1,
                                          padding="same",
                                          data_format=self._data_format,
                                          name="readout/conv4")
        

        #channels_last (NHWC) to channels_first (NCHW)
        if self._data_format == "channels_first":

            readout_output = tf.transpose(readout_output, (0, 2, 3, 1))

        self._output = readout_output

    def _upsample(self, stack, shape, factor):
        """This function resizes the input to a desired shape via the
           bilinear upsampling method.
        Args:
            stack (tensor, float32): A 4D tensor with the function input.
            shape (tensor, int32): A 1D tensor with the reference shape.
            factor (scalar, int): An integer denoting the upsampling factor.
        Returns:
            tensor, float32: A 4D tensor that holds the activations after
                             bilinear upsampling of the input.
        """

        if self._data_format == "channels_first":
            stack = tf.transpose(stack, (0, 2, 3, 1))

        stack = tf.image.resize_bilinear(stack, (shape[self._dims_axis[0]] * factor,
                                                 shape[self._dims_axis[1]] * factor))

        if self._data_format == "channels_first":
            stack = tf.transpose(stack, (0, 3, 1, 2))

        return stack

    def _gaussian_filter(self, r_output):
        """Convolving with a gaussian filter to regularize the predictions
        """
        x_zeros = np.zeros((21, 21))

        center = int(np.floor(21 / 2.))

        x_zeros[center, center] = 1
        y = gaussian_filter(x_zeros, sigma=7)
        min_y = np.min(y)
        new_y = y - min_y
        max_y = np.max(new_y)
        y = new_y / (max_y * 1.0)
        y = np.expand_dims(y, axis=2)
        gauss_blur_filter = np.expand_dims(y, axis=3)

        gb_filter_max = np.max(gauss_blur_filter)
        gauss_blur_filter = gauss_blur_filter / gb_filter_max
        gauss_blur_filter = tf.Variable(initial_value=tf.cast(gauss_blur_filter, tf.float32), trainable=True, name='gauss_blur_filter')

        # apply gaussian blur
        blurred_output= tf.nn.conv2d(tf.cast(r_output, tf.float32),
                                              gauss_blur_filter, [1, 1, 1, 1], padding='SAME')

        self._output = blurred_output 

    def _center_bias(self, blurred_out, center_bias):
        """Adding center fixation prior to the predictions
        """

        center_bias = np.expand_dims(center_bias, axis=0)
        center_bias = np.expand_dims(center_bias, axis=3)

        blurred_out_copy = tf.identity(blurred_out)

        center_bias = tf.image.resize_nearest_neighbor(center_bias, tf.shape(tf.squeeze(blurred_out_copy)))

        #output_biased = tf.divide(blurred_out, center_bias)
        output_biased = tf.add(blurred_out, center_bias)

        self._output =  output_biased
 
    def _normalize(self, maps, eps=1e-7):
        """This function normalizes the output values to a range
           between 0 and 1 per saliency map.
        Args:
            maps (tensor, float32): A 4D tensor that holds the model output.
            eps (scalar, float, optional): A small factor to avoid numerical
                                           instabilities. Defaults to 1e-7.
        """

        min_per_image = tf.reduce_min(maps, axis=(1, 2, 3), keep_dims=True)
        maps -= min_per_image

        max_per_image = tf.reduce_max(maps, axis=(1, 2, 3), keep_dims=True)
        maps = tf.divide(maps, eps + max_per_image, name="output")


        self._output = maps

            
    def _softmax(self, maps, eps=1e-10):
        """This function normalizes the output values to a range
           between 0 and 1 per saliency map, using softmax
        Args:
            maps (tensor, float32): A 4D tensor that holds the model output.
            eps (scalar, float, optional): A small factor to avoid numerical
                                           instabilities. Defaults to 1e-7.
        """
        sum_maps=tf.math.reduce_sum(tf.math.exp(maps) , keepdims=True)

        softmax_output =tf.divide( tf.math.exp(maps) ,  eps + sum_maps , name="output")


        self._output = softmax_output

    def _pretraining(self):
        """The first 32 variables of the model here are based on the VGG19
           network. Therefore, their names are matched to the ones of the
           pretrained VGG19 checkpoint for correct initialization.
        """#tf.get_variable_scope().set_use_resource(True)

        for var in tf.global_variables()[:32]:
            key = var.name
            #key = var.name.split("/", 1)[1]
            key = key.replace("kernel:0", "weights")
            key = key.replace("bias:0", "biases")
            #print(key)
            self._mapping[key] = var

    def forward(self, images, center_bias):
        """Public method to forward RGB images through the whole network
           architecture and retrieve the resulting output.
        Args:
            images (tensor, float32): A 4D tensor that holds the values of the
                                      raw input images.
        Returns:
            tensor, float32: A 4D tensor that holds the values of the
                             predicted saliency maps.
        """ 
        self._encoder(images)
        self._readout(self._output)
        #self._gaussian_filter(self._output)
        #self._center_bias(self._output, center_bias)
        #self._softmax(self._output)
        self._normalize(self._output)


        return self._output


    def train(self, ground_truth_map, predicted_maps, learning_rate, loss_f): #x, y, loss_f):
        """Public method to define the loss function and optimization
           algorithm for training the model.
        Args:
            ground_truth (tensor, float32): A 4D tensor with the ground truth.
            predicted_maps (tensor, float32): A 4D tensor with the predictions.
            learning_rate (scalar, float): Defines the learning rate.
        Returns:
            object: The optimizer element used to train the model.
            tensor, float32: A 0D tensor that holds the averaged error.
        """
        if loss_f=='kld':
          error = loss.kld(ground_truth_map, predicted_maps)

        #elif loss_f=='logl':
          #error = loss.log_likelihoods( x , y , 'train', predicted_maps)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        optimizer = optimizer.minimize(error)

        return optimizer, error


    def save(self, saver, sess, dataset, path, device):
        """This saves a model checkpoint to disk and creates
           the folder if it doesn't exist yet.
        Args:
            saver (object): An object for saving the model.
            sess (object): The current TF training session.
            path (str): The path used for saving the model.
            device (str): Represents either "cpu" or "gpu".
        """

        os.makedirs(path, exist_ok=True)

        saver.save(sess, path + "model_%s_%s.ckpt" % (dataset, device),
                   write_meta_graph=False, write_state=False)

    def restore(self, sess, dataset, paths, device):
        """This function allows continued training from a prior checkpoint and
           training from scratch with the pretrained VGG16 weights. In case the
           dataset is either CAT2000 or MIT1003, a prior checkpoint based on
           the SALICON dataset is required.
        Args:
            sess (object): The current TF training session.
            dataset ([type]): The dataset used for training.
            paths (dict, str): A dictionary with all path elements.
            device (str): Represents either "cpu" or "gpu".
        Returns:
            object: A saver object for saving the model.
        """

        model_name = "model_%s_%s" % (dataset, device)
        salicon_name = "model_salicon_%s" % device
        vgg19_name = "vgg19_normalized"

        ext1 = ".ckpt.data-00000-of-00001"
        ext2 = ".ckpt.index"

        saver = tf.train.Saver()

        if os.path.isfile(paths["latest"] + model_name + ext1) and \
           os.path.isfile(paths["latest"] + model_name + ext2):
            saver.restore(sess, paths["latest"] + model_name + ".ckpt")
        elif dataset in ("mit1003", "cat2000", "dutomron",
                         "pascals", "osie", "fiwi"):
            if os.path.isfile(paths["best"] + salicon_name + ext1) and \
               os.path.isfile(paths["best"] + salicon_name + ext2):
                saver.restore(sess, paths["best"] + salicon_name + ".ckpt")
            else:
                raise FileNotFoundError("Train model on SALICON first")
        else:

            self._pretraining()
            
            loader = tf.train.Saver(self._mapping)            
            #print_tensors_in_checkpoint_file(file_name=paths["weights"] + vgg19_name , tensor_name='', all_tensors=True)
            loader.restore(sess, paths["weights"] + vgg19_name )

            
        return saver

    def optimize(self, sess, dataset, path, device):
        """The best performing model is frozen, optimized for inference
           by removing unneeded training operations, and written to disk.
        Args:
            sess (object): The current TF training session.
            path (str): The path used for saving the model.
            device (str): Represents either "cpu" or "gpu".
        """

        model_name = "model_%s_%s" % (dataset, device)
        model_path = path + model_name

        tf.train.write_graph(sess.graph.as_graph_def(),
                             path, model_name + ".pbtxt")

        freeze_graph.freeze_graph(model_path + ".pbtxt", "", False,
                                  model_path + ".ckpt", "output",
                                  "save/restore_all", "save/Const:0",
                                  model_path + ".pb", True, "")


        os.remove(model_path + ".pbtxt")

        graph_def = tf.GraphDef()

        with tf.gfile.Open(model_path + ".pb", "rb") as file:
            graph_def.ParseFromString(file.read())

        transforms = ["remove_nodes(op=Identity)",
                      "merge_duplicate_nodes",
                      "strip_unused_nodes",
                      "fold_constants(ignore_errors=true)"]

        optimized_graph_def = TransformGraph(graph_def,
                                             ["input"],
                                             ["output"],
                                             transforms)

        tf.train.write_graph(optimized_graph_def,
                             logdir=path,
                             as_text=False,
                             name=model_name + ".pb")

**Print the training history and progress of the model:**

In [ ]:
import os
import time
from datetime import timedelta

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np


class History:
    """This class represents the training history of a model. It can load the
       prior history when training continues, keeps track of the training and
       validation error, and finally plots them as a curve after each epoch.
    """

    def __init__(self, n_train_batches, n_valid_batches,
                 dataset, path, device):
        self.train_history = []
        self.valid_history = []

        self._prior_epochs = 0

        self._train_error = 0
        self._valid_error = 0

        self._n_train_batches = n_train_batches
        self._n_valid_batches = n_valid_batches

        self._path = path
        self._id = (dataset, device)

        self._get_prior_history()

    def _get_prior_history(self):
        if os.path.isfile(self._path + "train_%s_%s.txt" % self._id):
            with open(self._path + "train_%s_%s.txt" % self._id, "r") as file:
                for line in file.readlines():
                    self.train_history.append(float(line))

        if os.path.isfile(self._path + "valid_%s_%s.txt" % self._id):
            with open(self._path + "valid_%s_%s.txt" % self._id, "r") as file:
                for line in file.readlines():
                    self.valid_history.append(float(line))

        self.prior_epochs = len(self.train_history)

    def update_train_step(self, train_error):
        self._train_error += train_error

    def update_valid_step(self, valid_error):
        self._valid_error += valid_error

    def get_mean_train_error(self, reset=True):
        mean_train_error = self._train_error / self._n_train_batches

        if reset:
            self._train_error = 0

        return mean_train_error

    def get_mean_valid_error(self, reset=True):
        mean_valid_error = self._valid_error / self._n_valid_batches

        if reset:
            self._valid_error = 0

        return mean_valid_error

    def save_history(self):
        mean_train_loss = self.get_mean_train_error(False)
        mean_valid_loss = self.get_mean_valid_error(False)

        self.train_history.append(mean_train_loss)
        self.valid_history.append(mean_valid_loss)

        os.makedirs(self._path, exist_ok=True)

        with open(self._path + "train_%s_%s.txt" % self._id, "a") as file:
            file.write("%f\n" % self.train_history[-1])

        with open(self._path + "valid_%s_%s.txt" % self._id, "a") as file:
            file.write("%f\n" % self.valid_history[-1])

        if len(self.train_history) > 1:
            axes = plt.figure().gca()

            x_range = np.arange(1, len(self.train_history) + 1)

            plt.plot(x_range, self.train_history, label="train", linewidth=2)
            plt.plot(x_range, self.valid_history, label="valid", linewidth=2)

            plt.legend()
            plt.xlabel("epochs")
            plt.ylabel("error")

            locations = plticker.MultipleLocator(base=1.0)
            axes.xaxis.set_major_locator(locations)

            plt.savefig(self._path + "curve_%s_%s.png" % self._id)
            plt.close()


class Progbar:
    """This class represents a progress bar for the terminal that visualizes
       the training progress for each epoch, estimated time of accomplishment,
       and then summarizes the training and validation loss together with the
       elapsed time.
    """

    def __init__(self, n_train_data, n_train_batches,
                 batch_size, n_epochs, prior_epochs):
        self._train_time = 0
        self._valid_time = 0

        self._start_time = time.time()

        self._batch_size = batch_size

        self._n_train_data = n_train_data
        self._n_train_batches = n_train_batches

        self._target_epoch = str(n_epochs + prior_epochs).zfill(2)
        self._current_epoch = str(prior_epochs + 1).zfill(2)

    def _flush(self):
        self._train_time = 0
        self._valid_time = 0

        self._start_time = time.time()

        current_epoch_int = int(self._current_epoch) + 1
        self._current_epoch = str(current_epoch_int).zfill(2)

    def update_train_step(self, current_batch):
        current_batch += 1

        self._train_time = time.time() - self._start_time
        batch_train_time = self._train_time / current_batch

        eta = (self._n_train_batches - current_batch) * batch_train_time
        eta = str(timedelta(seconds=np.ceil(eta)))

        progress_line = "=" * (20 * current_batch // self._n_train_batches)

        current_instance = current_batch * self._batch_size
        current_instance = np.clip(current_instance, 0, self._n_train_data)

        progress_frac = "%i/%i" % (current_instance, self._n_train_data)

        information = (self._current_epoch, self._target_epoch,
                       progress_line, progress_frac, eta)

        progbar_output = "Epoch %s/%s [%-20s] %s (ETA: %s)" % information

        print(progbar_output, end="\r", flush=True)

    def update_valid_step(self):
        self._valid_time = time.time() - self._start_time - self._train_time

    def write_summary(self, mean_train_loss, mean_valid_loss):
        train_time = str(timedelta(seconds=np.ceil(self._train_time)))
        valid_time = str(timedelta(seconds=np.ceil(self._valid_time)))

        train_information = (mean_train_loss, train_time)
        valid_information = (mean_valid_loss, valid_time)

        train_output = "\n\tTrain loss: %.6f (%s)" % train_information
        valid_output = "\tValid loss: %.6f (%s)" % valid_information

        print(train_output, flush=True)
        print(valid_output, flush=True)

        self._flush()


**Defining the Loss Function:**

In [ ]:
class loss():

  def kld(y_true, y_pred, eps=1e-7):
      """This function computes the Kullback-Leibler divergence between ground
        truth saliency maps and their predictions. Values are first divided by
        their sum for each image to yield a distribution that adds to 1.
      Args:
          y_true (tensor, float32): A 4d tensor that holds the ground truth
                                    saliency maps with values between 0 and 255.
          y_pred (tensor, float32): A 4d tensor that holds the predicted saliency
                                    maps with values between 0 and 1.
          eps (scalar, float, optional): A small factor to avoid numerical
                                        instabilities. Defaults to 1e-7.
      Returns:
          tensor, float32: A 0D tensor that holds the averaged error.
      """

      sum_per_image = tf.reduce_sum(y_true, axis=(1, 2, 3), keep_dims=True)
      y_true /= eps + sum_per_image

      sum_per_image = tf.reduce_sum(y_pred, axis=(1, 2, 3), keep_dims=True)
      y_pred /= eps + sum_per_image

      loss = y_true * tf.log(eps + y_true / (eps + y_pred))
      loss = tf.reduce_mean(tf.reduce_sum(loss, axis=(1, 2, 3)))

      return loss

  def log_likelihoods(x ,y , mode, predicted_maps, verbose=False):     

      loss = -(tf.reduce_sum(tf.log(tf.gather(tf.squeeze(predicted_maps),(y,x)))))/len(x)

      return loss

**Training and Testing the model on Salicon:**

ATTENTION: Please First Train the model for at least 2 epochs and then test the model.

In [32]:
import argparse
import os

import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
tf.logging.set_verbosity(tf.logging.ERROR)


def define_paths(current_path):
    """A helper function to define all relevant path elements for the
       locations of data, weights, and the results from either training
       or testing a model.
    Args:
        current_path (str): The path string of this script.
        args (object): A namespace object with values from command line.
    Returns:
        dict: A dictionary with all path elements.
    """

    data_path = current_path + "/"
    results_path =  current_path + "/results/"
    weights_path = current_path + "/weights/"

    history_path = results_path + "history/"
    images_path = results_path + "images/"
    ckpts_path = results_path + "ckpts/"

    best_path = ckpts_path + "best/"
    latest_path = ckpts_path + "latest/"

    paths = {
        "data": data_path,
        "history": history_path,
        "images": images_path,
        "best": best_path,
        "latest": latest_path,
        "weights": weights_path
    }

    return paths


def train_model(dataset, paths, device, center_bias_filter, loss_f):
    """The main function for executing network training. It loads the specified
       dataset iterator, saliency model, and helper classes. Training is then
       performed in a new session by iterating over all batches for a number of
       epochs. After validation on an independent set, the model is saved and
       the training history is updated.
    Args:
        dataset (str): Denotes the dataset to be used during training.
        paths (dict, str): A dictionary with all path elements.
        device (str): Represents either "cpu" or "gpu".
    """

    iterator = get_dataset_iterator("train", dataset, paths["data"])

    

    next_element, train_init_op, valid_init_op = iterator


    #input_images, ground_truth_maps,  x_train, y_train = next_element[:2]
    input_images, ground_truth_maps = next_element[:2]

    input_plhd = tf.placeholder_with_default(input_images,
                                             (None, None, None, 3),
                                             name="input")
    deepgaze2 = DeepGaze2()

    
    predicted_maps = deepgaze2.forward(input_plhd,center_bias_filter)

    #params1 = tf.convert_to_tensor(salicon_fixations_array_x)
    #params2 = tf.convert_to_tensor(salicon_fixations_array_y)

    #y_train = tf.gather(params1, indices)
    #x_train = tf.gather(params2, indices)

    #l = tf.convert_to_tensor(np.asarray(salicon_fixations_array_y)[np.asarray(indices])
    #                      , salicon_fixations_array_y

    optimizer, loss = deepgaze2.train(ground_truth_maps, predicted_maps,
                                    config.PARAMS["learning_rate"], loss_f)

    #x_valid = tf.convert_to_tensor(((salicon_fixations_train[salicon_fixations_train.n == np.asarray(indices)]).x).astype('int'))
    #y_valid = tf.convert_to_tensor(((salicon_fixations_train[salicon_fixations_train.n == np.asarray(indices)]).y).astype('int'))  
        
    n_train_data = eval(dataset.upper()).n_train
    n_valid_data = eval(dataset.upper()).n_valid

    n_train_batches = int(np.ceil(n_train_data / config.PARAMS["batch_size"]))
    n_valid_batches = int(np.ceil(n_valid_data / config.PARAMS["batch_size"]))

    history = History(n_train_batches,
                            n_valid_batches,
                            dataset,
                            paths["history"],
                            device)

    progbar = Progbar(n_train_data,
                            n_train_batches,
                            config.PARAMS["batch_size"],
                            config.PARAMS["n_epochs"],
                            history.prior_epochs)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = deepgaze2.restore(sess, dataset, paths, device)

        print(">> Start training on %s..." % dataset.upper())

        for epoch in range(config.PARAMS["n_epochs"]):
            sess.run(train_init_op)

            for batch in range(n_train_batches):
                _, error = sess.run([optimizer, loss])

                #print(sess.run(indices))
                #print(sess.run(l))

                history.update_train_step(error)
                progbar.update_train_step(batch)

            sess.run(valid_init_op)

            for batch in range(n_valid_batches):
                

                error = sess.run(loss)

                history.update_valid_step(error)
                progbar.update_valid_step()

            deepgaze2.save(saver, sess, dataset, paths["latest"], device)

            history.save_history()

            progbar.write_summary(history.get_mean_train_error(),
                                  history.get_mean_valid_error())

            if history.valid_history[-1] == min(history.valid_history):
                deepgaze2.save(saver, sess, dataset, paths["best"], device)
                deepgaze2.save(saver, sess, dataset, paths["best"], device)
                deepgaze2.optimize(sess, dataset, paths["best"], device)

                print("\tBest model!", flush=True)


def test_model(dataset, paths, device):
    """The main function for executing network testing. It loads the specified
       dataset iterator and optimized saliency model. By default, when no model
       checkpoint is found locally, the pretrained weights will be downloaded.
       Testing only works for models trained on the same device as specified in
       the config file.
    Args:
        dataset (str): Denotes the dataset that was used during training.
        paths (dict, str): A dictionary with all path elements.
        device (str): Represents either "cpu" or "gpu".
    """

    iterator = get_dataset_iterator("test", dataset, paths["data"])

    next_element, init_op = iterator

    input_images, original_shape, file_path = next_element

    graph_def = tf.GraphDef()

    model_name = "model_%s_%s.pb" % (dataset, device)

    if os.path.isfile(paths["best"] + model_name):
        with tf.gfile.Open(paths["best"] + model_name, "rb") as file:
            graph_def.ParseFromString(file.read())


        with tf.gfile.Open(paths["weights"] + model_name, "rb") as file:
            graph_def.ParseFromString(file.read())

    [predicted_maps] = tf.import_graph_def(graph_def,
                                           input_map={"input": input_images},
                                           return_elements=["output:0"])

    jpeg = postprocess_saliency_map(predicted_maps[0],
                                         original_shape[0])

    print(">> Start testing with %s %s model..." % (dataset.upper(), device))

    with tf.Session() as sess:
        sess.run(init_op)

        while True:
            try:
                output_file, path = sess.run([jpeg, file_path])
            except tf.errors.OutOfRangeError:
                break

            path = path[0][0].decode("utf-8")

            filename = os.path.basename(path)
            filename = os.path.splitext(filename)[0]
            filename += ".jpeg"

            os.makedirs(paths["images"], exist_ok=True)

            with open(paths["images"] + filename, "wb") as file:
                file.write(output_file)


def main():
    """The main function reads the command line arguments, invokes the
       creation of appropriate path variables, and starts the training
       or testing procedure for a model.
    """

    phases_list = ["train", "test"]
    datasets_list = ["salicon", "mit1003"]


    phase = input('Do you want to train or test? (enter train/test)')

    dataset = input('Which dataset do you want to use? (enter salicon/mit1003) ')
   
    loss_f = 'kld'
    current_path = "/content"
    
    #default_data_path = current_path + "data"

    paths = define_paths(current_path)

    tf.reset_default_graph() 

    # compute center bias filter
    center_bias_filter = MIT_log_baseline.astype('float32')

    if phase =='train':

      train_model(dataset, paths, config.PARAMS["device"] , center_bias_filter, loss_f)

    else: 
      test_model(dataset, paths, config.PARAMS["device"])

if __name__ == "__main__":
    
    main()

Do you want to train or test? (enter train/test)train
Which dataset do you want to use? (enter salicon/mit1003) salicon
<DatasetV1Adapter shapes: ((?, ?, ?, 3), (?, ?, ?, 1), (?, 2), ((?,), (?,))), types: (tf.float32, tf.float32, tf.int32, (tf.string, tf.string))>
>> Start training on SALICON...

	Train loss: 0.307299 (0:48:31)
	Valid loss: 0.895291 (0:01:48)
	Best model!

	Train loss: 0.251487 (0:48:32)
	Valid loss: 0.865506 (0:01:48)
	Best model!


KeyboardInterrupt: ignored